RSI 지표 과매수(70)시 매도 청산,  RSI 지표 과매도(30)시 매수 진입

In [ ]:
!pip install backtrader
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import backtrader as bt
import math
import yfinance as yf

In [ ]:
class RSI(bt.Strategy):
    params = dict(
        rsi_period=14,  # RSI 기간 설정
        rsi_low=30,  # RSI low 설정
        rsi_high=70  # RSI high 설정
    )
    def __init__(self):
        # RSI 지표 정의
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        # 매수 신호 추적 변수
        self.order = None

    def next(self):
        if self.order:  # 만약 주문이 있으면 대기
            return
        # 포지션이 없을 때(시장 미진입 상태)
        if not self.position:
            # RSI < 30이면 매수 진입 시도
            if self.rsi[-1] <= self.params.rsi_low < self.rsi[0]:  # RSI가 rsi_low보다 낮은지 확인하고 매수
                order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
                self.buy(size=order_size)

        # 포지션이 있을 때(시장 진입 상태)
        else:
            if self.rsi[-1] >= self.p.rsi_high > self.rsi[0]:  # RSI가 rsi_high보다 높은지 확인하고 매도
                self.close()

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


In [ ]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()
    cerebro.addstrategy(RSI)
    cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
    cerebro.broker.setcash(10_000_000)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-01')
    # 열 이름 전처리
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


Starting Portfolio Value: 10000000.00


[*********************100%***********************]  1 of 1 completed


2020-03-03 [매수 주문 실행] 종목:  	 수량: 1116 	 가격: 8865.39
2020-03-28 [매도 주문 실행] 종목:  	 수량: -1116 	 가격: 6467.25
2020-09-05 [매수 주문 실행] 종목:  	 수량: 684 	 가격: 10512.53
2020-09-21 [매도 주문 실행] 종목:  	 수량: -684 	 가격: 10934.93
2021-04-27 [매수 주문 실행] 종목:  	 수량: 137 	 가격: 54030.30
2021-08-11 [매도 주문 실행] 종목:  	 수량: -137 	 가격: 45599.70
2021-09-22 [매수 주문 실행] 종목:  	 수량: 153 	 가격: 40677.95
2021-10-23 [매도 주문 실행] 종목:  	 수량: -153 	 가격: 60694.63
None 주문이 거부되었습니다. 종목:  	 수량: 161 	 가격: 0.00
2021-11-28 [매수 주문 실행] 종목:  	 수량: 168 	 가격: 54813.02
2022-02-12 [매도 주문 실행] 종목:  	 수량: -168 	 가격: 42412.30
2022-02-25 [매수 주문 실행] 종목:  	 수량: 186 	 가격: 38333.75
2022-04-03 [매도 주문 실행] 종목:  	 수량: -186 	 가격: 45859.13
2022-04-20 [매수 주문 실행] 종목:  	 수량: 204 	 가격: 41501.75
2022-08-10 [매도 주문 실행] 종목:  	 수량: -204 	 가격: 23162.90
2022-09-03 [매수 주문 실행] 종목:  	 수량: 236 	 가격: 19969.72
2022-10-28 [매도 주문 실행] 종목:  	 수량: -236 	 가격: 20287.96
2022-11-11 [매수 주문 실행] 종목:  	 수량: 271 	 가격: 17583.25
2022-12-15 [매도 주문 실행] 종목:  	 수량: -271 	 가격: 17813.64
2022-12-30 

수익률이 매우 좋지 않다. 이번에는 종목 두 개(BTC, ETH)로 백테스팅 해 볼 예정

In [ ]:
class RSI_MULTI(bt.Strategy):
    params = dict(
        rsi_period=14,  # RSI 기간 설정
        rsi_low=30,  # RSI low 설정
        rsi_high=70  # RSI high 설정
    )

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.p.rsi_period)
        self.ind = dict()
        for i, v in enumerate(self.datas):
            self.ind[v] = dict()

    def next(self):
        for i, v in enumerate(self.datas):
            if self.getposition(v).size == 0:  # 포지션이 없을 경우
                if self.rsi[-1] <= self.p.rsi_low < self.rsi[0]:  # RSI가 rsi_low보다 낮은지 확인하고 매수
                    # 진입 가능 포지션 계산 (현금 / 현재가격)
                    if self.broker.get_cash() == self.broker.get_value():
                        # 진입한 종목이 없을 시 현재 보유한 현금 중 절반을 매수(수수료 때문에 0.99을 곱해줌)
                        order_size = math.floor(self.broker.get_cash() / v.close[0] / 2 * 0.99)
                    else:
                        # 진입한 종목이 있을 경우 나머지 현금으로 모두 매수(수수료 때문에 0.99을 곱해줌)
                        order_size = math.floor(self.broker.get_cash() / v.close[0] * 0.99)
                    self.buy(data=v, size=order_size)
            elif self.rsi[-1] >= self.p.rsi_high > self.rsi[0]:  # RSI가 rsi_high보다 높은지 확인하고 매도
                self.close(v)

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')




In [ ]:
if __name__ == "__main__":
    cerebro = bt.Cerebro()
    cerebro.broker.setcommission(commission=0.003)  # 0.3% 수수료 성정
    cerebro.broker.setcash(10_000_000)
    cerebro.addstrategy(RSI_MULTI)
    print("시작 포트폴리오 가격: %.2f" % cerebro.broker.get_value())

    # 여러 티커 설정
    tickers = ['BTC-USD', 'ETH-USD']
    for ticker in tickers:
        data = yf.download(ticker, start='2020-01-01', end='2024-12-01')
        # 컬럼 이름 정리
        data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
        # Backtrader 데이터 피드로 변환
        data_bt = bt.feeds.PandasData(dataname=data)
        # Cerebro에 해당 티커 추가
        # name 파라미터로 구분 이름을 붙일 수도 있음(예: name=ticker)
        cerebro.adddata(data_bt, name=ticker)

    # 백테스트 실행
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

시작 포트폴리오 가격: 10000000.00


2020-03-03 [매수 주문 실행] 종목: BTC-USD 	 수량: 558 	 가격: 8865.39
2020-03-03 [매수 주문 실행] 종목: ETH-USD 	 수량: 21468 	 가격: 230.52
2020-03-28 [매도 주문 실행] 종목: BTC-USD 	 수량: -558 	 가격: 6467.25
2020-03-28 [매도 주문 실행] 종목: ETH-USD 	 수량: -21468 	 가격: 134.03
2020-09-05 [매수 주문 실행] 종목: BTC-USD 	 수량: 308 	 가격: 10512.53
2020-09-05 [매수 주문 실행] 종목: ETH-USD 	 수량: 8339 	 가격: 388.04
2020-09-21 [매도 주문 실행] 종목: BTC-USD 	 수량: -308 	 가격: 10934.93
2020-09-21 [매도 주문 실행] 종목: ETH-USD 	 수량: -8339 	 가격: 371.40
2021-04-27 [매수 주문 실행] 종목: BTC-USD 	 수량: 59 	 가격: 54030.30
2021-04-27 [매수 주문 실행] 종목: ETH-USD 	 수량: 1268 	 가격: 2534.03
2021-08-11 [매도 주문 실행] 종목: BTC-USD 	 수량: -59 	 가격: 45599.70
2021-08-11 [매도 주문 실행] 종목: ETH-USD 	 수량: -1268 	 가격: 3142.83
2021-09-22 [매수 주문 실행] 종목: BTC-USD 	 수량: 81 	 가격: 40677.95
2021-09-22 [매수 주문 실행] 종목: ETH-USD 	 수량: 1204 	 가격: 2763.21
2021-10-23 [매도 주문 실행] 종목: BTC-USD 	 수량: -81 	 가격: 60694.63
2021-10-23 [매도 주문 실행] 종목: ETH-USD 	 수량: -1204 	 가격: 3971.36
2021-11-26 [매수 주문 실행] 종목: BTC-USD 	 수량: 84 	 가격: 58960.2

오히려 손해를 봤다.